In [6]:
import os, sys
import pickle
import pandas as pd
import numpy as np

# 피크 사이 wave를 모두 같은 length로 만들기 위한 함수
def linear_connection(list, idx):
    int_idx = int(idx)
    return list[int_idx] + (list[int_idx+1] - list[int_idx]) * (idx - int_idx)



### input 설정
LEN_INPUT = 10 # input 10s
LEN_PER_NRS = 120 # vital length for each NRS
OVERLAP = 5
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수


# vital data 저장 경로
preprocess_path = '../../cranberry2/Preprocessing/'
vital_path = preprocess_path + 'NRS_vital_pickle_unzip/NRS_vital_pickle'
f_vital_list = os.listdir(vital_path)


# 전처리 정보를 담을 Dataframe
column_list = ['file_path'] + [str(i+1) for i in range(n_aug)] #+ ['NRS']
df_preprocess = pd.DataFrame(columns = column_list)


SRATE, f_num = 100, 4000
for f_vital in f_vital_list[4000:6000]:
    f_num += 1
    
    print('###Input', f_num,'/ '+str(len(f_vital_list))+': '+f_vital+'###')
    
    # ppg, ecg peaks 불러오기
    # peaks가 없는 경우는 ECG나 PPG data가 없는 case들
    if not os.path.exists(preprocess_path+'cache/PPG_peaks/'+f_vital):
        print('no existing PPG peaks: ', f_vital)
        continue
    if not os.path.exists(preprocess_path+'cache/ECG_peaks/'+f_vital):
        print('no existing ECG peaks: ', f_vital)
        continue
    
    # vital data 불러오기    
    df_vital = pickle.load(open(vital_path+'/'+f_vital, 'rb')).reset_index()
    
    #dataframe에 새로운 행 만들기
    df_preprocess.loc[f_num-1,'file_path'] = f_vital
        
    ppg_min, ppg_peak = pickle.load(open(preprocess_path+'cache/PPG_peaks/'+f_vital, 'rb'))
    ecg_peak = pickle.load(open(preprocess_path+'cache/ECG_peaks/'+f_vital, 'rb'))
    
    ppg_min, ppg_peak = np.array([ppg_min]), np.array([ppg_peak])
    ecg_peak= np.array([ecg_peak])
    
    
    # 10초 단위로 끊기
    for i in range(n_aug):
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000
        
        seg_ppg, seg_ecg = [np.nan for j in range(LEN_INPUT*SRATE)], [np.nan for j in range(LEN_INPUT*SRATE)]
        df_vital_input = df_vital.loc[start_idx:end_idx-1]
        seg_ppg[0:len(df_vital_input)] = df_vital_input['Pleth'].tolist()
        seg_ecg[0:len(df_vital_input)] = df_vital_input['ECG'].tolist()
    
        ### 1. 결측치 처리 ###              
        # df.isnull().sum() 하면 더 간단하게 가능하나 애초에 NRS에 해당하는 vital data가 120초 보다 짧은 경우
        nan_ppg_list = np.isnan(seg_ppg)
        nan_ecg_list = np.isnan(seg_ecg)
        nan_ppg_perc = np.sum(nan_ppg_list) / LEN_INPUT / SRATE
        nan_ecg_perc = np.sum(nan_ecg_list) / LEN_INPUT / SRATE
        
        # ECG, PPG 둘다 결측치인 부분
        nan_both_perc = 0
        for j in range(len(seg_ppg)):
            if nan_ppg_list[j] and  nan_ecg_list[j]:
                nan_both_perc += 1
        nan_both_perc /= (LEN_INPUT*SRATE)
            
        # segment의 결측치 비율 정보
        nan_info = [nan_ppg_perc, nan_ecg_perc, nan_both_perc]
        
        # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
        if nan_ppg_perc > 0.3 or nan_ecg_perc > 0.3 or nan_both_perc > 0.2:
            df_preprocess.loc[f_num-1,str(i+1)] = (False, nan_info, [-1, -1])
            continue
            
        

        ### 2. Noise 처리 ###
        # 10초 segment 내의 ppg, ecg peak idx
        #seg_ppg_min = ppg_min[(start_idx<=np.array(ppg_min)) & (np.array(ppg_min)<end_idx)]
        idx_ppg_peak = ppg_peak[(start_idx<=ppg_peak) & (ppg_peak<end_idx)] - start_idx
        idx_ecg_peak = ecg_peak[(start_idx<=ecg_peak) & (ecg_peak<end_idx)] - start_idx
        
        # peak가 5개 이하는 noise가 많이 낀 상황 (10초 구간 중 peak가 7초 이상 없으면 문제 -> 즉 peak 개수 범위는 7/2 ~ 7/0.4)
        # 따라서 peak가 7초 이상 있어야하고 이때 최소 peak 개수는 3.5개 (20초면 7이하)
        # peak 개수가 기준 미달이면 noise 계산 자세히 할 필요없이 False - 이 경우의 noise_info는 -2로 처리
        if len(idx_ppg_peak)<=4 or len(idx_ecg_peak)<=4:
            df_preprocess.loc[f_num-1,str(i+1)] = (False, nan_info, [-2, -2])
            continue

            
        # 10초 segment 내의 ppg, ecg peak value
        #print(len(seg_ppg), idx_ppg_peak)
        val_ppg_peak = [seg_ppg[k] for k in idx_ppg_peak]
        val_ecg_peak = [seg_ecg[k] for k in idx_ecg_peak]
        
        # peak와 peak 사이 interval에 대한 noise 여부 -> 따라서 길이는 peak - 1
        bool_noise_ppg = [False for k in range(len(idx_ppg_peak)-1)]
        bool_noise_ecg = [False for k in range(len(idx_ecg_peak)-1)]
        
        #  2.1 peak 간격 이상한 noise (HR 30~150 -> HBI 0.4s ~ 2s로 SRATE 곱해주면 40~200)
        for k in range(len(bool_noise_ppg)):
            if not 40 < idx_ppg_peak[k+1] - idx_ppg_peak[k] < 200:
                bool_noise_ppg[k] = True
        for k in range(len(bool_noise_ecg)):
            if not 40 < idx_ecg_peak[k+1] - idx_ecg_peak[k] < 200:
                bool_noise_ecg[k] = True
                
        # 2.2 모양 이상한 noise
        # wave interval into same length(2s(200))
        len_wave = 200
        norm_seg_ppg, norm_seg_ecg = [], []

        for k in range(len(bool_noise_ppg)):
            len_interval_ppg = idx_ppg_peak[k+1] - idx_ppg_peak[k]
            
            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ppg.append([linear_connection(seg_ppg[idx_ppg_peak[k]:idx_ppg_peak[k+1]+1], n/len_wave*len_interval_ppg) for n in range(len_wave)])
        
        for k in range(len(bool_noise_ecg)):
            len_interval_ecg = idx_ecg_peak[k+1] - idx_ecg_peak[k]
            
            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ecg.append([linear_connection(seg_ecg[idx_ecg_peak[k]:idx_ecg_peak[k+1]+1], n/len_wave*len_interval_ecg) for n in range(len_wave)])
          
        
        # wave interval 사이 correlation 계산 - PPG
        mean_wave_ppg = np.nanmean(norm_seg_ppg, axis = 0)
        mean_wave_ppg = pd.DataFrame(mean_wave_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ppg = pd.DataFrame(norm_seg_ppg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ppg)):
            if np.corrcoef(norm_seg_ppg[k], mean_wave_ppg)[0,1] < 0.9:
                bool_noise_ppg[k] = True
        noise_ppg_perc = np.sum(bool_noise_ppg) / len(bool_noise_ppg)
        
        # wave interval 사이 correlation 계산 - ECG                
        mean_wave_ecg = np.nanmean(norm_seg_ecg, axis = 0)
        mean_wave_ecg = pd.DataFrame(mean_wave_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ecg = pd.DataFrame(norm_seg_ecg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ecg)):
            if np.corrcoef(norm_seg_ecg[k], mean_wave_ecg)[0,1] < 0.9:
                bool_noise_ecg[k] = True
        noise_ecg_perc = np.sum(bool_noise_ecg) / len(bool_noise_ecg)
        
        # segment의 noise 비율 정보
        noise_info = [noise_ppg_perc, noise_ecg_perc]
        
        # segment를 input으로 써도 되는지
        if nan_ppg_perc < 0.3 and nan_ecg_perc < 0.3 and nan_both_perc < 0.2 and noise_ppg_perc < 0.3 and noise_ecg_perc < 0.3:
            bool_pass = True
        else:
            bool_pass = False
       
        # 이 segment의 정보를 dataframe에 저장
        df_preprocess.loc[f_num-1,str(i+1)] = (bool_pass, nan_info, noise_info)        

    if f_num%1000 == 0:
        print('dumping cache of d_preprocess -', f_num, '/ 8318')
        pickle.dump(df_preprocess, open('cache/preprocess2/input1/df_preprocess_pacu_4000-6000', 'wb'))
        
print('dumping cache of d_preprocess -', f_num, '/ 8318')
pickle.dump(df_preprocess, open('cache/preprocess2/input1/df_preprocess_pacu_4000-6000', 'wb'))


###Input 4001 / 8318: 6.0,4225,PACU1_1_200303_131837.csv###
###Input 4002 / 8318: 4.0,2122,PACU1_5_190904_145810.csv###
###Input 4003 / 8318: 5.0,4188,PACU1_8_200302_172103.csv###
###Input 4004 / 8318: 5.0,5266,PACU1_5_200520_101422.csv###
###Input 4005 / 8318: 4.0,8252,PACU1_8_201027_100019.csv###
###Input 4006 / 8318: 4.0,231,PACU1_5_190503_083054.csv###
###Input 4007 / 8318: 4.0,3321,PACU1_5_200129_103728.csv###
###Input 4008 / 8318: 5.0,431,PACU1_5_190514_142925.csv###
###Input 4009 / 8318: 5.0,1942,PACU1_4_190829_110553.csv###
###Input 4010 / 8318: 8.0,8282,PACU1_5_201027_192835.csv###
###Input 4011 / 8318: 3.0,4284,PACU1_6_200305_101952.csv###
###Input 4012 / 8318: 5.0,3252,PACU1_3_200122_175909.csv###
###Input 4013 / 8318: 7.0,7800,PACU1_11_201005_142838.csv###
###Input 4014 / 8318: 9.0,1344,PACU1_5_190718_182050.csv###
###Input 4015 / 8318: 3.0,2465,PACU1_7_191018_152522.csv###
###Input 4016 / 8318: 4.0,6471,PACU1_11_200715_132358.csv###
###Input 4017 / 8318: 8.0,2369,PACU1_4_1

###Input 4137 / 8318: 5.0,7990,PACU1_5_201013_204758.csv###
###Input 4138 / 8318: 5.0,7012,PACU1_2_200812_164026.csv###
###Input 4139 / 8318: 4.0,3637,PACU1_1_200211_103021.csv###
###Input 4140 / 8318: 8.0,1861,PACU1_10_190827_113303.csv###
###Input 4141 / 8318: 5.0,2327,PACU1_5_190910_183522.csv###
###Input 4142 / 8318: 5.0,3014,PACU1_9_200115_104905.csv###
###Input 4143 / 8318: 4.0,3813,PACU1_4_200218_154434.csv###
###Input 4144 / 8318: 4.0,6191,PACU1_2_200630_154304.csv###
###Input 4145 / 8318: 2.0,3887,PACU1_1_200219_211754.csv###
###Input 4146 / 8318: 8.0,1269,PACU1_3_190717_113028.csv###
###Input 4147 / 8318: 2.0,8199,PACU1_7_201023_154625.csv###
###Input 4148 / 8318: 8.0,8334,PACU1_8_201029_144720.csv###
###Input 4149 / 8318: 5.0,6436,PACU1_5_200714_165941.csv###
###Input 4150 / 8318: 6.0,852,PACU1_3_190612_013632.csv###
###Input 4151 / 8318: 10.0,7571,PACU1_1_200921_215157.csv###
###Input 4152 / 8318: 7.0,6013,PACU1_6_200618_171233.csv###
###Input 4153 / 8318: 4.0,1860,PACU1_2_

###Input 4274 / 8318: 5.0,529,PACU1_1_190516_203320.csv###
###Input 4275 / 8318: 4.0,4973,PACU1_10_200507_132540.csv###
###Input 4276 / 8318: 4.0,4506,PACU1_4_200313_170104.csv###
###Input 4277 / 8318: 5.0,5381,PACU1_8_200526_172636.csv###
###Input 4278 / 8318: 3.0,802,PACU1_12_190607_134925.csv###
###Input 4279 / 8318: 4.0,493,PACU1_1_190516_090804.csv###
###Input 4280 / 8318: 4.0,2498,PACU1_3_191022_084657.csv###
###Input 4281 / 8318: 7.0,3688,PACU1_6_200213_142252.csv###
###Input 4282 / 8318: 4.0,2131,PACU1_8_190904_162533.csv###
###Input 4283 / 8318: 5.0,7892,PACU1_6_201008_103600.csv###
###Input 4284 / 8318: 4.0,6312,PACU1_4_200707_102804.csv###
###Input 4285 / 8318: 5.0,3538,PACU1_3_200206_163111.csv###
###Input 4286 / 8318: 5.0,7043,PACU1_4_200814_021212.csv###
###Input 4287 / 8318: 5.0,2091,PACU1_4_190904_012006.csv###
###Input 4288 / 8318: 5.0,3562,PACU1_7_200207_114542.csv###
###Input 4289 / 8318: 7.0,4827,PACU1_11_200326_144213.csv###
###Input 4290 / 8318: 5.0,7650,PACU1_5_2

###Input 4411 / 8318: 4.0,3473,PACU1_5_200205_094457.csv###
###Input 4412 / 8318: 7.0,7482,PACU1_9_200917_114802.csv###
###Input 4413 / 8318: 2.0,7570,PACU1_4_200921_193519.csv###
###Input 4414 / 8318: 3.0,5312,PACU1_1_200521_214609.csv###
###Input 4415 / 8318: 2.0,5250,PACU1_6_200519_163717.csv###
###Input 4416 / 8318: 2.0,5705,PACU1_2_200608_090708.csv###
###Input 4417 / 8318: 3.0,6055,PACU1_6_200622_171038.csv###
###Input 4418 / 8318: 3.0,3030,PACU1_10_200115_134501.csv###
###Input 4419 / 8318: 8.0,8049,PACU1_9_201016_111800.csv###
###Input 4420 / 8318: 4.0,6752,PACU1_7_200729_084603.csv###
###Input 4421 / 8318: 3.0,1509,PACU1_6_190816_145600.csv###
###Input 4422 / 8318: 5.0,4566,PACU1_5_200317_171459.csv###
###Input 4423 / 8318: 3.0,6096,PACU1_6_200624_113822.csv###
###Input 4424 / 8318: 5.0,1998,PACU1_1_190830_133627.csv###
###Input 4425 / 8318: 6.0,4560,PACU1_11_200317_160936.csv###
###Input 4426 / 8318: 8.0,5060,PACU1_1_200511_190645.csv###
###Input 4427 / 8318: 2.0,4761,PACU1_1

###Input 4548 / 8318: 5.0,7320,PACU1_3_200910_131322.csv###
###Input 4549 / 8318: 2.0,1229,PACU1_10_190716_100507.csv###
###Input 4550 / 8318: 6.0,3974,PACU1_5_200221_163003.csv###
###Input 4551 / 8318: 3.0,4952,PACU1_3_200506_173310.csv###
###Input 4552 / 8318: 7.0,6229,PACU1_6_200701_175602.csv###
###Input 4553 / 8318: 8.0,5773,PACU1_10_200610_111202.csv###
###Input 4554 / 8318: 6.0,6733,PACU1_4_200728_103723.csv###
###Input 4555 / 8318: 6.0,4553,PACU1_8_200317_125607.csv###
###Input 4556 / 8318: 7.0,1208,PACU1_3_190715_150624.csv###
###Input 4557 / 8318: 2.0,5083,PACU1_3_200512_142935.csv###
###Input 4558 / 8318: 5.0,4216,PACU1_3_200303_105459.csv###
###Input 4559 / 8318: 8.0,6235,PACU1_3_200701_203516.csv###
###Input 4560 / 8318: 5.0,5364,PACU1_1_200526_123356.csv###
###Input 4561 / 8318: 3.0,3268,PACU1_1_200123_095225.csv###
###Input 4562 / 8318: 5.0,3958,PACU1_6_200221_141548.csv###
###Input 4563 / 8318: 9.0,2514,PACU1_7_191023_084609.csv###
###Input 4564 / 8318: 6.0,7576,PACU1_2

...

###Input 5414 / 8318: 7.0,6461,PACU1_6_200715_114620.csv###
###Input 5415 / 8318: 5.0,6668,PACU1_1_200723_211733.csv###
###Input 5416 / 8318: 3.0,7752,PACU1_6_200928_170228.csv###
###Input 5417 / 8318: 7.0,1231,PACU1_5_190716_105345.csv###
###Input 5418 / 8318: 3.0,2151,PACU1_12_190905_103501.csv###
###Input 5419 / 8318: 8.0,1644,PACU1_11_190820_164140.csv###
###Input 5420 / 8318: 5.0,3765,PACU1_1_200217_111202.csv###
###Input 5421 / 8318: 2.0,6024,PACU1_9_200619_112814.csv###
###Input 5422 / 8318: 8.0,1920,PACU1_7_190828_154822.csv###
###Input 5423 / 8318: 5.0,4830,PACU1_10_200326_163746.csv###
###Input 5424 / 8318: 8.0,2541,PACU1_2_191024_160232.csv###
###Input 5425 / 8318: 5.0,749,PACU1_2_190531_092313.csv###
###Input 5426 / 8318: 3.0,5093,PACU1_10_200512_174020.csv###
###Input 5427 / 8318: 3.0,6086,PACU1_2_200623_223241.csv###
###Input 5428 / 8318: 4.0,6092,PACU1_4_200624_094808.csv###
###Input 5429 / 8318: 3.0,2111,PACU1_3_190904_132112.csv###
###Input 5430 / 8318: 5.0,4267,PACU1_

###Input 5549 / 8318: 5.0,5179,PACU1_12_200515_132430.csv###
###Input 5550 / 8318: 5.0,1872,PACU1_4_190827_153927.csv###
###Input 5551 / 8318: 3.0,6310,PACU1_4_200707_102804.csv###
###Input 5552 / 8318: 3.0,3037,PACU1_6_200115_160853.csv###
###Input 5553 / 8318: 7.0,3121,PACU1_10_200117_165910.csv###
###Input 5554 / 8318: 5.0,7319,PACU1_3_200910_131322.csv###
###Input 5555 / 8318: 2.0,6892,PACU1_11_200805_165907.csv###
###Input 5556 / 8318: 5.0,7262,PACU1_3_200901_144509.csv###
###Input 5557 / 8318: 5.0,1640,PACU1_10_190820_163322.csv###
###Input 5558 / 8318: 8.0,5847,PACU1_2_200612_141022.csv###
###Input 5559 / 8318: 9.0,6535,PACU1_1_200716_174742.csv###
###Input 5560 / 8318: 7.0,7708,PACU1_3_200925_142027.csv###
###Input 5561 / 8318: 6.0,2164,PACU1_11_190905_114453.csv###
###Input 5562 / 8318: 7.0,6353,PACU1_3_200708_154355.csv###
###Input 5563 / 8318: 4.0,6635,PACU1_7_200722_141700.csv###
###Input 5564 / 8318: 7.0,6226,PACU1_7_200701_174342.csv###
###Input 5565 / 8318: 8.0,588,PACU1

###Input 5686 / 8318: 4.0,1378,PACU1_7_190719_133554.csv###
###Input 5687 / 8318: 5.0,4273,PACU1_7_200304_181914.csv###
###Input 5688 / 8318: 6.0,2785,PACU1_6_200106_160435.csv###
###Input 5689 / 8318: 2.0,4845,PACU1_3_200327_094439.csv###
###Input 5690 / 8318: 6.0,4692,PACU1_9_200320_114058.csv###
###Input 5691 / 8318: 6.0,3674,PACU1_1_200211_221449.csv###
###Input 5692 / 8318: 5.0,2869,PACU1_2_200109_101801.csv###
###Input 5693 / 8318: 5.0,4473,PACU1_9_200312_181328.csv###
###Input 5694 / 8318: 3.0,4815,PACU1_1_200326_110905.csv###
###Input 5695 / 8318: 2.0,7560,PACU1_1_200921_142848.csv###
###Input 5696 / 8318: 2.0,7072,PACU1_3_200818_092917.csv###
###Input 5697 / 8318: 5.0,2102,PACU1_5_190904_092222.csv###
###Input 5698 / 8318: 4.0,6498,PACU1_4_200716_085512.csv###
###Input 5699 / 8318: 5.0,6249,PACU1_12_200702_152130.csv###
###Input 5700 / 8318: 4.0,1820,PACU1_5_190823_120409.csv###
###Input 5701 / 8318: 6.0,4170,PACU1_10_200302_153314.csv###
###Input 5702 / 8318: 4.0,2337,PACU1_1

###Input 5823 / 8318: 4.0,3706,PACU1_3_200213_230122.csv###
###Input 5824 / 8318: 3.0,4323,PACU1_8_200306_120714.csv###
###Input 5825 / 8318: 6.0,4972,PACU1_10_200507_132540.csv###
###Input 5826 / 8318: 7.0,1513,PACU1_3_190816_152001.csv###
###Input 5827 / 8318: 5.0,4162,PACU1_8_200302_145600.csv###
###Input 5828 / 8318: 7.0,2991,PACU1_11_200114_131810.csv###
###Input 5829 / 8318: 8.0,4021,PACU1_1_200224_172545.csv###
###Input 5830 / 8318: 4.0,3185,PACU1_7_200121_165655.csv###
###Input 5831 / 8318: 5.0,7275,PACU1_1_200902_114221.csv###
###Input 5832 / 8318: 5.0,3670,PACU1_7_200211_205050.csv###
###Input 5833 / 8318: 3.0,4469,PACU1_10_200312_144115.csv###
###Input 5834 / 8318: 5.0,7082,PACU1_1_200818_125223.csv###
###Input 5835 / 8318: 8.0,6703,PACU1_4_200724_232157.csv###
###Input 5836 / 8318: 5.0,6753,PACU1_7_200729_093231.csv###
###Input 5837 / 8318: 3.0,5024,PACU1_9_200508_155023.csv###
###Input 5838 / 8318: 8.0,2398,PACU1_5_190917_141536.csv###
###Input 5839 / 8318: 3.0,2331,PACU1_

###Input 5960 / 8318: 3.0,3477,PACU1_1_200205_100911.csv###
###Input 5961 / 8318: 8.0,456,PACU1_5_190515_120702.csv###
###Input 5962 / 8318: 2.0,7318,PACU1_10_200910_124041.csv###
###Input 5963 / 8318: 4.0,3182,PACU1_1_200121_153741.csv###
###Input 5964 / 8318: 6.0,5695,PACU1_5_200605_183256.csv###
###Input 5965 / 8318: 3.0,3700,PACU1_1_200213_204423.csv###
###Input 5966 / 8318: 7.0,7150,PACU1_10_200820_154629.csv###
###Input 5967 / 8318: 5.0,4575,PACU1_7_200318_091050.csv###
###Input 5968 / 8318: 2.0,5372,PACU1_9_200526_152346.csv###
###Input 5969 / 8318: 3.0,1292,PACU1_1_190718_083748.csv###
###Input 5970 / 8318: 3.0,4468,PACU1_10_200312_144115.csv###
###Input 5971 / 8318: 2.0,4694,PACU1_1_200320_115359.csv###
###Input 5972 / 8318: 4.0,6547,PACU1_7_200717_112147.csv###
###Input 5973 / 8318: 5.0,7059,PACU1_8_200817_121744.csv###
###Input 5974 / 8318: 5.0,5469,PACU1_5_200528_183608.csv###
###Input 5975 / 8318: 5.0,5733,PACU1_12_200609_100437.csv###
###Input 5976 / 8318: 4.0,4044,PACU1_

In [7]:
# input 설정
LEN_INPUT = 10 # input 10s
LEN_PER_NRS = 120 # vital length for each NRS
OVERLAP = 5
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수


# vital data 저장 경로
vital_path = '../../cranberry2/Preprocessing/preop_vital/preop'
f_vital_list = os.listdir(vital_path)

# 전처리 정보를 담을 Dataframe
column_list = ['file_path'] + [str(i+1) for i in range(n_aug)] #+ ['NRS']
df_preprocess = pd.DataFrame(columns = column_list)


SRATE, f_num = 100, 2000
for f_vital in f_vital_list[2000:3000]:
    f_num += 1
    
    print('###Input', f_num,'/ '+str(len(f_vital_list))+': '+f_vital+'###')
    
    # ppg, ecg peaks 불러오기
    # peaks가 없는 경우는 ECG나 PPG data가 없는 case들
    if not os.path.exists('../../cranberry2/Preprocessing/cache/PPG_peaks/'+f_vital):
        print('no existing PPG peaks: ', f_vital)
        continue
    if not os.path.exists('../../cranberry2/Preprocessing/cache/ECG_peaks/'+f_vital):
        print('no existing ECG peaks: ', f_vital)
        continue
    
    # vital data 불러오기    
    df_vital = pickle.load(open(vital_path+'/'+f_vital, 'rb')).reset_index()
    
    #dataframe에 새로운 행 만들기
    df_preprocess.loc[f_num-1,'file_path'] = f_vital
        
    ppg_min, ppg_peak = pickle.load(open('../../cranberry2/Preprocessing/cache/PPG_peaks/'+f_vital, 'rb'))
    ecg_peak = pickle.load(open('../../cranberry2/Preprocessing/cache/ECG_peaks/'+f_vital, 'rb'))
    
    ppg_min, ppg_peak = np.array([ppg_min]), np.array([ppg_peak])
    ecg_peak= np.array([ecg_peak])
    
    
    # 10초 단위로 끊기
    for i in range(n_aug):
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000
        
        seg_ppg, seg_ecg = [np.nan for j in range(LEN_INPUT*SRATE)], [np.nan for j in range(LEN_INPUT*SRATE)]
        df_vital_input = df_vital.loc[start_idx:end_idx-1]
        seg_ppg[0:len(df_vital_input)] = df_vital_input['Pleth'].tolist()
        seg_ecg[0:len(df_vital_input)] = df_vital_input['ECG'].tolist()
    
        ### 1. 결측치 처리 ###              
        # df.isnull().sum() 하면 더 간단하게 가능하나 애초에 NRS에 해당하는 vital data가 120초 보다 짧은 경우
        nan_ppg_list = np.isnan(seg_ppg)
        nan_ecg_list = np.isnan(seg_ecg)
        nan_ppg_perc = np.sum(nan_ppg_list) / LEN_INPUT / SRATE
        nan_ecg_perc = np.sum(nan_ecg_list) / LEN_INPUT / SRATE
        
        # ECG, PPG 둘다 결측치인 부분
        nan_both_perc = 0
        for j in range(len(seg_ppg)):
            if nan_ppg_list[j] and  nan_ecg_list[j]:
                nan_both_perc += 1
        nan_both_perc /= (LEN_INPUT*SRATE)
            
        # segment의 결측치 비율 정보
        nan_info = [nan_ppg_perc, nan_ecg_perc, nan_both_perc]
        
        # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
        if nan_ppg_perc > 0.3 or nan_ecg_perc > 0.3 or nan_both_perc > 0.2:
            df_preprocess.loc[f_num-1,str(i+1)] = (False, nan_info, [-1, -1])
            continue
            
        

        ### 2. Noise 처리 ###
        # 10초 segment 내의 ppg, ecg peak idx
        #seg_ppg_min = ppg_min[(start_idx<=np.array(ppg_min)) & (np.array(ppg_min)<end_idx)]
        idx_ppg_peak = ppg_peak[(start_idx<=ppg_peak) & (ppg_peak<end_idx)] - start_idx
        idx_ecg_peak = ecg_peak[(start_idx<=ecg_peak) & (ecg_peak<end_idx)] - start_idx
        
        # peak가 5개 이하는 noise가 많이 낀 상황 (10초 구간 중 peak가 7초 이상 없으면 문제 -> 즉 peak 개수 범위는 7/2 ~ 7/0.4)
        # 따라서 peak가 7초 이상 있어야하고 이때 최소 peak 개수는 3.5개
        # peak 개수가 기준 미달이면 noise 계산 자세히 할 필요없이 False - 이 경우의 noise_info는 -2로 처리
        if len(idx_ppg_peak)<=4 or len(idx_ecg_peak)<=4:
            df_preprocess.loc[f_num-1,str(i+1)] = (False, nan_info, [-2, -2])
            continue

            
        # 10초 segment 내의 ppg, ecg peak value
        #print(len(seg_ppg), idx_ppg_peak)
        val_ppg_peak = [seg_ppg[k] for k in idx_ppg_peak]
        val_ecg_peak = [seg_ecg[k] for k in idx_ecg_peak]
        
        # peak와 peak 사이 interval에 대한 noise 여부 -> 따라서 길이는 peak - 1
        bool_noise_ppg = [False for k in range(len(idx_ppg_peak)-1)]
        bool_noise_ecg = [False for k in range(len(idx_ecg_peak)-1)]
        
        #  2.1 peak 간격 이상한 noise (HR 30~150 -> HBI 0.4s ~ 2s로 SRATE 곱해주면 40~200)
        for k in range(len(bool_noise_ppg)):
            if not 40 < idx_ppg_peak[k+1] - idx_ppg_peak[k] < 200:
                bool_noise_ppg[k] = True
        for k in range(len(bool_noise_ecg)):
            if not 40 < idx_ecg_peak[k+1] - idx_ecg_peak[k] < 200:
                bool_noise_ecg[k] = True
                
        # 2.2 모양 이상한 noise
        # wave interval into same length(2s(200))
        len_wave = 200
        norm_seg_ppg, norm_seg_ecg = [], []

        for k in range(len(bool_noise_ppg)):
            len_interval_ppg = idx_ppg_peak[k+1] - idx_ppg_peak[k]
            
            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ppg.append([linear_connection(seg_ppg[idx_ppg_peak[k]:idx_ppg_peak[k+1]+1], n/len_wave*len_interval_ppg) for n in range(len_wave)])
        
        for k in range(len(bool_noise_ecg)):
            len_interval_ecg = idx_ecg_peak[k+1] - idx_ecg_peak[k]
            
            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ecg.append([linear_connection(seg_ecg[idx_ecg_peak[k]:idx_ecg_peak[k+1]+1], n/len_wave*len_interval_ecg) for n in range(len_wave)])
          
        
        # wave interval 사이 correlation 계산 - PPG
        mean_wave_ppg = np.nanmean(norm_seg_ppg, axis = 0)
        mean_wave_ppg = pd.DataFrame(mean_wave_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ppg = pd.DataFrame(norm_seg_ppg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ppg)):
            if np.corrcoef(norm_seg_ppg[k], mean_wave_ppg)[0,1] < 0.9:
                bool_noise_ppg[k] = True
        noise_ppg_perc = np.sum(bool_noise_ppg) / len(bool_noise_ppg)
        
        # wave interval 사이 correlation 계산 - ECG                
        mean_wave_ecg = np.nanmean(norm_seg_ecg, axis = 0)
        mean_wave_ecg = pd.DataFrame(mean_wave_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ecg = pd.DataFrame(norm_seg_ecg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ecg)):
            if np.corrcoef(norm_seg_ecg[k], mean_wave_ecg)[0,1] < 0.9:
                bool_noise_ecg[k] = True
        noise_ecg_perc = np.sum(bool_noise_ecg) / len(bool_noise_ecg)
        
        # segment의 noise 비율 정보
        noise_info = [noise_ppg_perc, noise_ecg_perc]
        
        # segment를 input으로 써도 되는지
        if nan_ppg_perc < 0.3 and nan_ecg_perc < 0.3 and nan_both_perc < 0.2 and noise_ppg_perc < 0.3 and noise_ecg_perc < 0.3:
            bool_pass = True
        else:
            bool_pass = False
       
        # 이 segment의 정보를 dataframe에 저장
        df_preprocess.loc[f_num-1,str(i+1)] = (bool_pass, nan_info, noise_info)        

    if f_num%1000 == 0:
        print('dumping cache of d_preprocess -', f_num, '/ 3888')
        pickle.dump(df_preprocess, open('cache/preprocess2/input1/df_preprocess_preop_2000-3000', 'wb'))
        
print('dumping cache of d_preprocess -', f_num, '/ 3888')
pickle.dump(df_preprocess, open('cache/preprocess2/input1/df_preprocess_preop_2000-3000', 'wb'))


###Input 2001 / 3888: 0.0,568,PACU1_2_190523_084951.csv###
###Input 2002 / 3888: 0.0,4113,PACU1_7_200228_161438.csv###
###Input 2003 / 3888: 0.0,3414,PACU1_4_200203_103851.csv###
###Input 2004 / 3888: 0.0,2033,PACU1_4_190902_133811.csv###
###Input 2005 / 3888: 0.0,526,PACU1_2_190516_172118.csv###
###Input 2006 / 3888: 0.0,3518,PACU1_2_200205_211323.csv###
###Input 2007 / 3888: 0.0,772,PACU1_11_190531_161211.csv###
###Input 2008 / 3888: 0.0,3900,PACU1_1_200220_130022.csv###
###Input 2009 / 3888: 0.0,6502,PACU1_6_200716_102317.csv###
###Input 2010 / 3888: 0.0,1170,PACU1_6_190712_113935.csv###
###Input 2011 / 3888: 0.0,6297,PACU1_4_200706_172613.csv###
###Input 2012 / 3888: 0.0,6033,PACU1_6_200619_135031.csv###
###Input 2013 / 3888: 0.0,6691,PACU1_3_200724_153849.csv###
###Input 2014 / 3888: 0.0,1563,PACU1_10_190819_161953.csv###
###Input 2015 / 3888: 0.0,6858,PACU1_3_200804_140134.csv###
###Input 2016 / 3888: 0.0,4158,PACU1_10_200302_140215.csv###
###Input 2017 / 3888: 0.0,1524,PACU1_3_1

###Input 2138 / 3888: 0.0,3313,PACU1_9_200129_100629.csv###
###Input 2139 / 3888: 0.0,7442,PACU1_9_200916_131909.csv###
###Input 2140 / 3888: 0.0,4112,PACU1_11_200228_154436.csv###
###Input 2141 / 3888: 0.0,4747,PACU1_9_200324_113352.csv###
###Input 2142 / 3888: 0.0,1115,PACU1_2_190710_171004.csv###
###Input 2143 / 3888: 0.0,2397,PACU1_3_190917_122637.csv###
###Input 2144 / 3888: 0.0,45,PACU1_3_190413_001656.csv###
###Input 2145 / 3888: 0.0,5213,PACU1_3_200517_165101.csv###
###Input 2146 / 3888: 0.0,569,PACU1_4_190523_090239.csv###
###Input 2147 / 3888: 0.0,1434,PACU1_7_190723_112630.csv###
###Input 2148 / 3888: 0.0,4258,PACU1_3_200304_144520.csv###
###Input 2149 / 3888: 0.0,3143,PACU1_3_200120_125809.csv###
###Input 2150 / 3888: 0.0,7717,PACU1_8_200925_174811.csv###
###Input 2151 / 3888: 0.0,4338,PACU1_10_200306_144514.csv###
###Input 2152 / 3888: 0.0,8128,PACU1_3_201020_211912.csv###
###Input 2153 / 3888: 0.0,4972,PACU1_10_200507_132540.csv###
###Input 2154 / 3888: 0.0,6565,PACU1_7_2

###Input 2275 / 3888: 0.0,4448,PACU1_1_200311_173037.csv###
###Input 2276 / 3888: 0.0,5931,PACU1_8_200616_143026.csv###
###Input 2277 / 3888: 0.0,2461,PACU1_2_191018_152208.csv###
###Input 2278 / 3888: 0.0,2432,PACU1_9_191017_114003.csv###
###Input 2279 / 3888: 0.0,4173,PACU1_3_200302_160235.csv###
###Input 2280 / 3888: 0.0,8365,PACU1_11_201030_143744.csv###
###Input 2281 / 3888: 0.0,3009,PACU1_6_200115_090424.csv###
###Input 2282 / 3888: 0.0,3690,PACU1_7_200213_160304.csv###
###Input 2283 / 3888: 0.0,5065,PACU1_6_200512_092601.csv###
###Input 2284 / 3888: 0.0,6520,PACU1_9_200716_154909.csv###
###Input 2285 / 3888: 0.0,7330,PACU1_2_200910_181432.csv###
###Input 2286 / 3888: 0.0,591,PACU1_3_190523_183320.csv###
###Input 2287 / 3888: 0.0,773,PACU1_5_190531_165639.csv###
###Input 2288 / 3888: 0.0,649,PACU1_6_190528_063147.csv###
###Input 2289 / 3888: 0.0,4419,PACU1_1_200311_085737.csv###
###Input 2290 / 3888: 0.0,4219,PACU1_5_200303_124705.csv###
###Input 2291 / 3888: 0.0,5422,PACU1_1_200

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


###Input 2324 / 3888: 0.0,4646,PACU1_5_200318_183733.csv###
###Input 2325 / 3888: 0.0,7687,PACU1_2_200925_091922.csv###
###Input 2326 / 3888: 0.0,1764,PACU1_1_190821_212040.csv###
###Input 2327 / 3888: 0.0,4815,PACU1_1_200326_110905.csv###
###Input 2328 / 3888: 0.0,8049,PACU1_9_201016_111800.csv###
###Input 2329 / 3888: 0.0,1066,PACU1_1_190709_150603.csv###
no existing PPG peaks:  0.0,1066,PACU1_1_190709_150603.csv
###Input 2330 / 3888: 0.0,3466,PACU1_1_200204_210634.csv###
###Input 2331 / 3888: 0.0,6586,PACU1_4_200720_095702.csv###
###Input 2332 / 3888: 0.0,7944,PACU1_3_201010_010844.csv###
###Input 2333 / 3888: 0.0,4991,PACU1_3_200508_022437.csv###
###Input 2334 / 3888: 0.0,7553,PACU1_2_200921_123408.csv###
###Input 2335 / 3888: 0.0,6740,PACU1_1_200728_122610.csv###
###Input 2336 / 3888: 0.0,4982,PACU1_7_200507_173837.csv###
###Input 2337 / 3888: 0.0,6199,PACU1_2_200701_084823.csv###
###Input 2338 / 3888: 0.0,6869,PACU1_7_200805_091134.csv###
###Input 2339 / 3888: 0.0,3497,PACU1_9_20

###Input 2460 / 3888: 0.0,3961,PACU1_11_200221_150904.csv###
###Input 2461 / 3888: 0.0,628,PACU1_8_190527_115843.csv###
###Input 2462 / 3888: 0.0,5645,PACU1_1_200604_153532.csv###
###Input 2463 / 3888: 0.0,8147,PACU1_7_201021_140619.csv###
###Input 2464 / 3888: 0.0,25,PACU1_2_190411_084556.csv###
###Input 2465 / 3888: 0.0,244,PACU1_1_190503_114505.csv###
###Input 2466 / 3888: 0.0,3551,PACU1_1_200207_091413.csv###
###Input 2467 / 3888: 0.0,7564,PACU1_6_200921_164714.csv###
###Input 2468 / 3888: 0.0,7868,PACU1_1_201007_152611.csv###
###Input 2469 / 3888: 0.0,3014,PACU1_9_200115_104905.csv###
###Input 2470 / 3888: 0.0,840,PACU1_1_190611_180631.csv###
###Input 2471 / 3888: 0.0,787,PACU1_8_190603_120856.csv###
###Input 2472 / 3888: 0.0,8248,PACU1_4_201027_095610.csv###
###Input 2473 / 3888: 0.0,7681,PACU1_1_200924_205336.csv###
###Input 2474 / 3888: 0.0,499,PACU1_5_190516_110638.csv###
###Input 2475 / 3888: 0.0,5677,PACU1_10_200605_130948.csv###
###Input 2476 / 3888: 0.0,4472,PACU1_3_200312

###Input 2596 / 3888: 0.0,214,PACU1_9_190502_154658.csv###
###Input 2597 / 3888: 0.0,3976,PACU1_5_200221_172749.csv###
###Input 2598 / 3888: 0.0,3524,PACU1_5_200206_105613.csv###
###Input 2599 / 3888: 0.0,4007,PACU1_7_200224_140725.csv###
###Input 2600 / 3888: 0.0,1473,PACU1_1_190726_164200.csv###
###Input 2601 / 3888: 0.0,6608,PACU1_11_200721_144619.csv###
###Input 2602 / 3888: 0.0,7785,PACU1_1_200929_211106.csv###
###Input 2603 / 3888: 0.0,914,PACU1_4_190703_160251.csv###
###Input 2604 / 3888: 0.0,5315,PACU1_5_200522_110849.csv###
###Input 2605 / 3888: 0.0,913,PACU1_2_190703_160110.csv###
###Input 2606 / 3888: 0.0,7347,PACU1_2_200911_113648.csv###
###Input 2607 / 3888: 0.0,8224,PACU1_1_201026_112949.csv###
###Input 2608 / 3888: 0.0,3017,PACU1_4_200115_112819.csv###
###Input 2609 / 3888: 0.0,5850,PACU1_9_200612_153815.csv###
###Input 2610 / 3888: 0.0,1153,PACU1_5_190711_085132.csv###
###Input 2611 / 3888: 0.0,5805,PACU1_3_200611_111510.csv###
###Input 2612 / 3888: 0.0,225,PACU1_3_1905

###Input 2732 / 3888: 0.0,207,PACU1_1_190502_133146.csv###
###Input 2733 / 3888: 0.0,1666,PACU1_3_190820_181030.csv###
###Input 2734 / 3888: 0.0,4668,PACU1_10_200319_135911.csv###
###Input 2735 / 3888: 0.0,6214,PACU1_6_200701_122322.csv###
###Input 2736 / 3888: 0.0,4412,PACU1_4_200310_164830.csv###
###Input 2737 / 3888: 0.0,4853,PACU1_2_200327_140627.csv###
###Input 2738 / 3888: 0.0,7616,PACU1_1_200923_114746.csv###
###Input 2739 / 3888: 0.0,5606,PACU1_11_200603_145538.csv###
###Input 2740 / 3888: 0.0,711,PACU1_3_190529_162322.csv###
###Input 2741 / 3888: 0.0,3832,PACU1_6_200219_090110.csv###
###Input 2742 / 3888: 0.0,2207,PACU1_3_190906_095537.csv###
###Input 2743 / 3888: 0.0,4633,PACU1_3_200318_155533.csv###
###Input 2744 / 3888: 0.0,2899,PACU1_5_200109_200741.csv###
###Input 2745 / 3888: 0.0,1341,PACU1_6_190718_172632.csv###
###Input 2746 / 3888: 0.0,8305,PACU1_1_201028_132155.csv###
###Input 2747 / 3888: 0.0,3868,PACU1_3_200219_154420.csv###
###Input 2748 / 3888: 0.0,4733,PACU1_4_2

###Input 2869 / 3888: 0.0,4895,PACU1_3_200331_011647.csv###
###Input 2870 / 3888: 0.0,86,PACU1_1_190419_195900.csv###
###Input 2871 / 3888: 0.0,5882,PACU1_4_200615_123313.csv###
###Input 2872 / 3888: 0.0,4987,PACU1_5_200507_222942.csv###
###Input 2873 / 3888: 0.0,2036,PACU1_5_190902_151743.csv###
###Input 2874 / 3888: 0.0,4227,PACU1_3_200303_154819.csv###
###Input 2875 / 3888: 0.0,6577,PACU1_11_200717_155747.csv###
no existing PPG peaks:  0.0,6577,PACU1_11_200717_155747.csv
###Input 2876 / 3888: 0.0,4316,PACU1_9_200306_104617.csv###
###Input 2877 / 3888: 0.0,4833,PACU1_4_200326_174721.csv###
###Input 2878 / 3888: 0.0,3295,PACU1_2_200128_121653.csv###
###Input 2879 / 3888: 0.0,3712,PACU1_4_200214_005121.csv###
###Input 2880 / 3888: 0.0,2110,PACU1_2_190904_125828.csv###
###Input 2881 / 3888: 0.0,7787,PACU1_2_200930_000526.csv###
###Input 2882 / 3888: 0.0,7257,PACU1_3_200901_123527.csv###
###Input 2883 / 3888: 0.0,4149,PACU1_7_200302_122709.csv###
###Input 2884 / 3888: 0.0,6397,PACU1_3_20